### Japanese Language Flashcard Generation

#### Overview
This notebook automates the creation of Japanese language Anki flashcards from textbook images. It combines OCR technology with large language model processing to extract, verify, and format vocabulary into ready-to-import flashcards.

#### Features
- Extract Japanese text from textbook images using OCR API
- Cross-reference extracted text with original images for accuracy
- Generate structured CSV flashcards with proper formatting
- Support for contextual vocabulary notes and usage examples

#### Workflow
1. **Image Upload**: Upload Japanese textbook page images  
2. **Text Extraction**: Use OCR API to extract text from the images  
3. **LLM Processing**: Send both the extracted text and original image to an LLM  
4. **Flashcard Generation**: Generate structured CSV data using specialized prompts from `LLM_Prompts.py`  
5. **Export**: Save the resulting flashcards in Anki-compatible CSV format  

#### Flashcard Format
The generated flashcards follow a specific CSV structure:
- **Kanji column**: Contains the word in Kanji (or Hiragana/Katakana if no Kanji exists)  
- **Furigana column**: Contains the phonetic reading of the word in Hiragana  
- **English_Translation_and_Notes column**: Contains both the English translation and any usage or contextual notes  

Example output:
```
"迷う [道に～]","まよう [みちに～]","lose one's way (e.g., get lost on the road)"  
"先輩","せんぱい","senior (student, colleague, etc.)"  
```

#### Benefits
- **Accuracy**: Cross-references OCR text with the original image to fix errors  
- **Context-Aware**: Preserves usage examples and contextual information  
- **Time-Saving**: Automates the tedious process of manual flashcard creation  
- **Customizable**: Prompts can be adjusted for different textbook formats  

#### Applications
- Creating comprehensive JLPT study materials  
- Building personal vocabulary decks from textbooks  
- Supplementing classroom learning with digital flashcards  
- Archiving vocabulary from various Japanese learning resources

In [ ]:
# Importing the required libraries
from unstract.llmwhisperer import LLMWhispererClientV2
import os
from dotenv import load_dotenv

# Defining variables
load_dotenv()
unstract_api_url = os.getenv("LLMWHISPERER_BASE_URL_V2")
unstract_api_key = os.getenv("LLMWHISPERER_API_KEY")
image_path = r"C:\Users\Rounak\OneDrive\Documents\Japanese Study\Flashcards - Textbook Photos and Text Files\MNN Intermediate 1\Chatpter 1\01page_1.jpg"

# Check if image paths exists
if not os.path.exists(image_path):
    print("Image path does not exist")
    exit()

# Extract text from the image using LLMWhisperer
client = LLMWhispererClientV2(base_url=unstract_api_url, api_key=unstract_api_key, logging_level="ERROR")  # Change the logging level to "ERROR" for production
result= client.whisper(file_path=image_path, wait_for_completion=True)
print(result["extraction"]["result_text"])

In [ ]:
import base64

image_path_example_1 = r"C:\Users\Rounak\OneDrive\Documents\Japanese Study\Flashcards - Textbook Photos and Text Files\MNN Beginner 1 & 2 - Useful Words & Information\Chapter 12.jpg"
image_path_example_2 = r"C:\Users\Rounak\OneDrive\Documents\Japanese Study\Flashcards - Textbook Photos and Text Files\MNN Intermediate 1\Chatpter 1\01page_1.jpg"

def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8') # Decode to string for text storage
        return encoded_string

encoded_string_example_1 = image_to_base64(image_path_example_1)
encoded_string_example_2 = image_to_base64(image_path_example_2)

# Write to a text file
with open("encoded_image_example_1.txt", "w") as text_file:
    text_file.write(encoded_string_example_1)

with open("encoded_image_example_2.txt", "w") as text_file:
    text_file.write(encoded_string_example_2)

In [ ]:
import base64
from images_base_64 import mnn_useful_words_chapter_12_example_1, mnn_intermediate_chapter_1_page_1_example_2

def load_image_from_base64(base64_string): # Helper function for loading base64 images
    try:
        image_bytes = base64.b64decode(base64_string)
        from io import BytesIO
        image = PIL.Image.open(BytesIO(image_bytes))
        return image
    except Exception as e:
        print(f"Error loading image from base64: {e}")
        return None

image_example_1 = load_image_from_base_64(mnn_useful_words_chapter_12_example_1)
image_example_1.show()